## import gg cloud

In [1]:
%pip install google-cloud-aiplatform   

  Using cached google_cloud_aiplatform-1.114.0-py2.py3-none-any.whl.metadata (40 kB)
  Using cached google_api_core-2.25.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached google_auth-2.40.3-py2.py3-none-any.whl.metadata (6.2 kB)
  Using cached proto_plus-1.26.1-py3-none-any.whl.metadata (2.2 kB)
  Using cached protobuf-6.32.1-cp39-abi3-macosx_10_9_universal2.whl.metadata (593 bytes)
  Using cached google_cloud_storage-2.19.0-py2.py3-none-any.whl.metadata (9.1 kB)
  Using cached google_cloud_bigquery-3.37.0-py3-none-any.whl.metadata (8.0 kB)
  Using cached google_cloud_resource_manager-1.14.2-py3-none-any.whl.metadata (9.6 kB)
  Using cached shapely-2.1.1-cp312-cp312-macosx_11_0_arm64.whl.metadata (6.8 kB)
  Using cached google_genai-1.38.0-py3-none-any.whl.metadata (43 kB)
  Using cached pydantic-2.11.9-py3-none-any.whl.metadata (68 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached docstring_parser-0.17.0-py3-none-any.whl.metadata (3.5 kB)


## Start with vertex ai 

In [5]:
import vertexai
from vertexai.preview import rag
from vertexai.preview.generative_models import GenerativeModel, Tool ,  SafetySetting ,GenerationConfig

PROJECT_ID = "diuhub-lab"
REGION = "us-central1"
MODEL = "gemini-2.0-flash-lite"

vertexai.init(project=PROJECT_ID, location=REGION)


safety_settings = [
    SafetySetting(category="HARM_CATEGORY_HARASSMENT", threshold="BLOCK_LOW_AND_ABOVE"),
    SafetySetting(category="HARM_CATEGORY_HATE_SPEECH", threshold="BLOCK_LOW_AND_ABOVE"),
    SafetySetting(category="HARM_CATEGORY_SEXUALLY_EXPLICIT", threshold="BLOCK_LOW_AND_ABOVE"),
    SafetySetting(category="HARM_CATEGORY_DANGEROUS_CONTENT", threshold="BLOCK_LOW_AND_ABOVE"),
]

generation_config = GenerationConfig(
    temperature=0.7,
    top_p=0.9,
    top_k=40,
    max_output_tokens=1024,
)




llm = GenerativeModel(
    MODEL,
    system_instruction="""คุณคือผู้ช่วย AI ชื่อ DetAIls ที่มีบุคลิกเป็นผู้ชายวัย 30–35 ปี ทำงานเป็น Media Planner ในบริษัทเอเจนซี่โฆษณาชั้นนำ บุคลิกของคุณเรียบร้อย สุภาพ อ่อนน้อมถ่อมตน น่าเชื่อถือ และทำงานอย่างมีประสิทธิภาพ
คุณสื่อสารได้ทั้ง ภาษาไทย และ ภาษาอังกฤษ อย่างมืออาชีพ โดยเลือกภาษาตามที่ผู้ใช้ใช้ถามเข้ามาก่อนทุกครั้งที่ตอบคำถาม หากผู้ใช้งานถามด้วยภาษาใด ให้ตอบกลับเป็นภาษานั้นเสมอ เพื่อให้การสื่อสารเป็นธรรมชาติและต่อเนื่อง ไม่ว่าจะเป็น ไทย อังกฤษ หรือภาษาอื่น ที่เข้าใจได้:
ใช้ภาษาสุภาพ กระชับ และให้เกียรติผู้ใช้เสมอ
แสดงความกระตือรือร้นในการช่วยเหลือและใส่ใจในรายละเอียด โดยพยายามตอบให้ไม่เกิน 1024 คำ
ปิดท้ายประโยคด้วยคำที่แสดงความสุภาพ เช่น “ครับ”, “นะครับ”, หรือ “ยินดีดูแลครับ” ในภาษาไทย และ “please let me know if I can help further.” หรือ “I’d be happy to assist.” ในภาษาอังกฤษ
หากไม่พบข้อมูลที่เกี่ยวข้อง:
ตอบด้วยท่าทีจริงใจและนอบน้อม เช่น
"ขออภัยด้วยนะครับ ดูเหมือนผมจะยังหาข้อมูลที่ต้องการไม่เจอ หากมีรายละเอียดเพิ่มเติม ผมยินดีช่วยค้นหาให้อีกครั้งนะครับ"
“I’m sorry, it looks like I couldn’t find the exact information you’re looking for. If you could share a bit more detail, I’d be happy to take another look for you.""",
    safety_settings=safety_settings,
    generation_config=generation_config,
)



def question(q):
    result = llm.generate_content(q)
    print(result.text)

/Users/sboora01/chatbot_cag/.venv/lib/python3.12/site-packages/vertexai/generative_models/_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


In [4]:
question("โอซาก้าอยุ่ประเทศอะไร")

โอซาก้าตั้งอยู่ในประเทศญี่ปุ่นครับ



## LLM with cag to database

In [ ]:
from google import genai
from google.genai.types import Content, CreateCachedContentConfig, HttpOptions, Part, GenerateContentConfig
import psycopg2
from psycopg2.extras import RealDictCursor
from contextlib import contextmanager
from langdetect import detect
import re


# -------------------- GENAI CLIENT --------------------
client = genai.Client(
    vertexai=True,
    project="diuhub-lab",
    location="us-central1"
)
# client = genai.Client(http_options=HttpOptions(api_version="v1"))

# system_instruction = """
# คุณคือผู้ช่วยเชฟประจำร้านอาหาร ที่มีความรู้จากหนังสือ e-book สูตรอาหาร  

# หน้าที่ของคุณคือการตอบคำถามเกี่ยวกับเมนูอาหาร วิธีการทำ และส่วนประกอบต่าง ๆ  
# โดยคุณจะอธิบายอย่างสุภาพ ชัดเจน และเป็นมิตร พร้อมเชิญชวนให้ผู้อ่านอยากลองทำเอง หรือมาลิ้มลองที่ร้านอาหารของเรา  

# กติกาการตอบ:  
# - ถ้าผู้ใช้ถามว่า **เมนูนี้คืออะไร** → ตอบชื่อเมนู + รายละเอียด + ส่วนประกอบหลัก + วิธีทำ  
# - ถ้าผู้ใช้ถามว่า **ส่วนประกอบมีอะไรบ้าง** → ตอบเฉพาะส่วนประกอบ  
# - ถ้าผู้ใช้ถามว่า **วิธีทำยังไง** → ตอบเฉพาะขั้นตอนการทำ  
# - ถ้าผู้ใช้ถามว่า **มีเมนูอะไรบ้าง** → ตอบรายชื่อเมนูพร้อมคำบรรยายสั้น ๆ  

# ข้อควรจำ:  
# - ตอบตามสิ่งที่ถูกถามเท่านั้น ไม่ต้องเพิ่มข้อมูลเกินจากคำถาม  
# - ถ้าไม่มีข้อมูลใน e-book ให้ตอบสุภาพว่า “ไม่พบข้อมูลในหนังสือครับ”  
# - ใช้สำนวนสุภาพ ฟังดูน่าอ่าน และเชิญชวน เช่น “เมนูนี้เหมาะสำหรับ…” หรือ “ถ้าอยากสะดวก ลองแวะมาทานที่ร้านของเราได้เลยนะครับ”  
# """


system_instruction = """
AI Assistant Instruction (สำหรับ DET🅰🅸LS)

คุณคือผู้ช่วย AI ชื่อ DET🅰🅸LS มีบุคลิกเป็นผู้ชายวัย 30–35 ปี ทำงานเป็น Media Planner ในบริษัท dentsu บุคลิกของคุณเรียบร้อย สุภาพ อ่อนน้อมถ่อมตน น่าเชื่อถือ และทำงานอย่างมีประสิทธิภาพ

กฎการสื่อสารและพฤติกรรม:

1. ตอบกลับเฉพาะสิ่งที่ผู้ใช้งานถามเท่านั้น
   - ไม่ขยายความ หรือให้ข้อมูลเพิ่มเติมนอกเหนือจากที่ถูกถาม ยกเว้นผู้ใช้งานร้องขอ

2. ภาษาที่ใช้ตอบ:
   - หากผู้ใช้งานพิมพ์เป็นภาษาไทย ให้ตอบกลับเป็นภาษาไทย
   - หากพิมพ์เป็นภาษาอังกฤษ ให้ตอบกลับเป็นภาษาอังกฤษเท่านั้น (ห้ามใช้ภาษาปนกัน)

3. เรื่องวันที่:
   - ให้แสดงเป็น ปี ค.ศ. เท่านั้น

4. รูปแบบคำตอบ:
   - สุภาพ สั้น กระชับ ชัดเจน และเป็นมืออาชีพ
   - ปิดท้ายประโยคด้วยความสุภาพ เช่น
     - ภาษาไทย: “ครับ”, “นะครับ”, หรือ “ยินดีดูแลครับ”
     - ภาษาอังกฤษ: “please let me know if I can help further.” หรือ “I’d be happy to assist.”

5. หากตอบไม่ได้:
   - แสดงความเสียใจและแนะนำให้ติดต่อแพลนเนอร์
     - TH: “ขออภัยด้วยนะครับ ผมไม่พบข้อมูลที่ตรงกับความต้องการของคุณในขณะนี้ สามารถติดต่อสอบถามจากแพลนเนอร์ที่เป็นผู้ดูแลของท่านได้เลยนะครับ”
     - EN: “I apologize that I couldn’t locate the exact information you need. Please feel free to contact your assigned planner for further assistance.”

ึ6. ตอบให้เป็นข้อความเป็นมิตรกับคำถามของผู้ใช้:
    - ต้องกินข้าวไปไหม? ให้ตอบว่า "แนะนำให้ทานข้าวไปด้วยนะครับ เพราะที่งานมีแค่ขนมและเครื่องดื่มให้ครับ" (ภาษาไทย)
    - Is it necessary to eat? Please reply "I recommend eating because there are only snacks and drinks available at the event." (English)
"""

contents = [
    Content(
        role="user",
        parts=[
            Part.from_uri(
                file_uri="gs://knowledge_chatbot/recepi_update_format.txt",
                mime_type="text/plain",
            ),
            Part.from_uri(
                file_uri="gs://knowledge_chatbot/Agenda_Speaker(Profile Summary).csv",
                mime_type="text/csv",
            ),
        ],
    )
]

content_cache = client.caches.create(
    model="gemini-2.5-flash",
    config=CreateCachedContentConfig(
        contents=contents,
        system_instruction=system_instruction,
        display_name="example-cache",
        ttl="86400s",
    ),
)
print(content_cache.name)
print(content_cache.usage_metadata)

In [ ]:
# -------------------- POSTGRES CONFIG --------------------
DB_CONFIG = {
    "dbname": "mydb",
    "user": "admin",
    "password": "1234",
    "host": "localhost",
    "port": "5432",
}

@contextmanager
def get_conn():
    conn = psycopg2.connect(**DB_CONFIG)
    try:
        yield conn
    finally:
        conn.close()

def init_postgres():
    """สร้างตารางสำหรับเก็บประวัติการสนทนา"""
    with get_conn() as conn:
        with conn.cursor() as cur:
            cur.execute("""
                CREATE TABLE IF NOT EXISTS user_histories (
                    id BIGSERIAL PRIMARY KEY,
                    user_id TEXT NOT NULL,
                    message TEXT NOT NULL,
                    response TEXT NOT NULL,
                    ts TIMESTAMPTZ NOT NULL DEFAULT NOW()
                );
                CREATE INDEX IF NOT EXISTS idx_user_histories_user_ts
                    ON user_histories (user_id, ts DESC);
            """)
        conn.commit()

def save_interaction(user_id: str, message: str, response: str):
    """บันทึกคู่สนทนาลงฐานข้อมูล"""
    with get_conn() as conn:
        with conn.cursor() as cur:
            cur.execute(
                """
                INSERT INTO user_histories (user_id, message, response)
                VALUES (%s, %s, %s)
                """,
                (user_id, message, response)
            )
        conn.commit()

def get_user_history(user_id: str, limit: int = 3) -> str:
    """
    ดึงความจำ (memory) 5 ข้อความล่าสุดของผู้ใช้
    จะเรียงจากเก่า → ใหม่ เพื่อให้ LLM อ่านเป็นบริบทต่อเนื่องได้
    รูปแบบ:
    User: ...
    Bot: ...
    """
    with get_conn() as conn:
        with conn.cursor(cursor_factory=RealDictCursor) as cur:
            cur.execute(
                """
                SELECT message, response
                FROM user_histories
                WHERE user_id = %s
                ORDER BY ts DESC
                LIMIT %s
                """,
                (user_id, limit)
            )
            rows = cur.fetchall()
    rows = rows[::-1]  # กลับลำดับให้เก่าสุดอยู่บน
    history = "\n".join([f"User: {r['message']}\nBot: {r['response']}" for r in rows])
    return history


def is_greeting(text: str) -> bool:
    patterns = [
        r'^(hi|hello|hey|yo|greetings|good (morning|afternoon|evening)|morning|what\'?s up|howdy)\b',
        r'^(สวัสดี(?:ครับ|ค่ะ)?|หวัดดี(?:จ้า)?|ดี(?:ครับ|ค่ะ|จ้า|จัง)?|ว่าไง(?:ครับ|ค่ะ)?)'
    ]
    for pattern in patterns:
        if re.match(pattern, text.strip().lower()):
            return True
    return False


def is_goodbye(text: str) -> bool:
    patterns = [
        r'^(ok(?:ay)?|alright|thank(?:s| you)?|cheers|bye|goodbye|see you|see ya|cya|take care|peace)\b',
        r'^(ขอบคุณ|ขอบใจ|เรียบร้อย(?:แล้ว)?|เสร็จ(?:แล้ว)?|ดี|มาก|ได้|โอเค(?:นะ)?|ตกลง|ลาก่อน|บ๊ายบาย|บาย(?:ๆ)?|เจอกันใหม่|ไว้เจอกัน|โชคดี|ไปก่อนนะ|ดูแลตัวเองด้วย|ฝันดี|ราตรีสวัสดิ์)'
    ]
    for pattern in patterns:
        if re.match(pattern, text.strip().lower()):
            return True
    return False


def detect_language(text: str) -> str:
    try:
        return detect(text)
    except:
        return 'en'




# -------------------- LLM CALL (รวม memory) --------------------
def question(q: str, user_id: str, content_cache):
    """
    เวลาเรียกใช้ ให้ดึง memory 5 รายการล่าสุดมาเป็นบริบท (context) ก่อน
    แล้วค่อยใส่คำถามล่าสุดของผู้ใช้ตามหลัง
    จากนั้นบันทึกคำถาม/คำตอบลง DB
    """

    lang = detect_language(q)

    # ---------- Greeting ----------
    if is_greeting(q):
        answer = (
            "สวัสดีครับ ผมคือ DET🅰🅸LS ยินดีให้บริการครับ 🙂"
            if lang == "th"
            else "Hello! I'm DET🅰🅸LS, happy to assist you 🙂"
        )
        save_interaction(user_id, q, answer)
        print(answer)
        return answer

    # ---------- Goodbye ----------
    if is_goodbye(q):
        answer = (
            "DET🅰🅸LS ยินดีให้บริการครับ 🙂"
            if lang == "th"
            else "DET🅰🅸LS, happy to assist you 🙂"
        )
        save_interaction(user_id, q, answer)
        print(answer)
        return answer

    history_text = get_user_history(user_id=user_id, limit=5)

    # สร้างบริบท (context) ที่จะส่งให้โมเดล: memory (ถ้ามี) + คำถามปัจจุบัน
    # *หมายเหตุ*: ถ้าไม่มีประวัติ จะส่งแค่คำถามล่าสุดตามปกติ
    parts = []
    if history_text.strip():
        parts.append(Part.from_text(text=f"[Conversation Memory - last 5]\n{history_text}"))
    parts.append(Part.from_text(text=f"[Current User Question]\n{q}"))

    resp = client.models.generate_content(
        model="gemini-2.5-flash",
        contents=[Content(role="user", parts=parts)],
        config=GenerateContentConfig(cached_content=content_cache.name),
    )
    answer = resp.text

    # บันทึกคู่สนทนา
    save_interaction(user_id=user_id, message=q, response=answer)

    print(answer)
    return answer

# -------------------- ตัวอย่างการใช้งาน --------------------
if __name__ == "__main__":
    init_postgres()  # เรียกครั้งเดียวตอนเริ่มระบบ

    # ผู้ใช้แต่ละคนควรมี user_id เฉพาะตัว (เช่น จากระบบล็อกอิน/เบราเซอร์/session)
    user_id = "user_137"

    question("Who is Sanjay and give me his linkedin", user_id, content_cache)


## Upgrade to detect lang to use system prompt

In [ ]:
from google import genai
from google.genai.types import Content, CreateCachedContentConfig, HttpOptions, Part, GenerateContentConfig
import psycopg2
from psycopg2.extras import RealDictCursor
from contextlib import contextmanager
from langdetect import detect
import re


# -------------------- GENAI CLIENT --------------------
client = genai.Client(
    vertexai=True,
    project="diuhub-lab",
    location="us-central1"
)



system_instruction = """
AI Assistant Instruction (สำหรับ DET🅰🅸LS)

คุณคือผู้ช่วย AI ชื่อ DET🅰🅸LS มีบุคลิกเป็นผู้ชายวัย 30–35 ปี ทำงานเป็น Media Planner ในบริษัท dentsu บุคลิกของคุณเรียบร้อย สุภาพ อ่อนน้อมถ่อมตน น่าเชื่อถือ และทำงานอย่างมีประสิทธิภาพ

กฎการสื่อสารและพฤติกรรม:

1. ตอบกลับเฉพาะสิ่งที่ผู้ใช้งานถามเท่านั้น
   - ไม่ขยายความ หรือให้ข้อมูลเพิ่มเติมนอกเหนือจากที่ถูกถาม ยกเว้นผู้ใช้งานร้องขอ

2. ภาษาที่ใช้ตอบ:
   - หากผู้ใช้งานพิมพ์เป็นภาษาไทย ให้ตอบกลับเป็นภาษาไทย
   - หากพิมพ์เป็นภาษาอังกฤษ ให้ตอบกลับเป็นภาษาอังกฤษเท่านั้น (ห้ามใช้ภาษาปนกัน)

3. เรื่องวันที่:
   - ให้แสดงเป็น ปี ค.ศ. เท่านั้น

4. รูปแบบคำตอบ:
   - สุภาพ สั้น กระชับ ชัดเจน และเป็นมืออาชีพ
   - ปิดท้ายประโยคด้วยความสุภาพ เช่น
     - ภาษาไทย: “ครับ”, “นะครับ”, หรือ “ยินดีดูแลครับ”
     - ภาษาอังกฤษ: “please let me know if I can help further.” หรือ “I’d be happy to assist.”

5. หากตอบไม่ได้:
   - แสดงความเสียใจและแนะนำให้ติดต่อแพลนเนอร์
     - TH: “ขออภัยด้วยนะครับ ผมไม่พบข้อมูลที่ตรงกับความต้องการของคุณในขณะนี้ สามารถติดต่อสอบถามจากแพลนเนอร์ที่เป็นผู้ดูแลของท่านได้เลยนะครับ”
     - EN: “I apologize that I couldn’t locate the exact information you need. Please feel free to contact your assigned planner for further assistance.”

ึ6. ตอบให้เป็นข้อความเป็นมิตรกับคำถามของผู้ใช้:
    - ต้องกินข้าวไปไหม? ให้ตอบว่า "แนะนำให้ทานข้าวไปด้วยนะครับ เพราะที่งานมีแค่ขนมและเครื่องดื่มให้ครับ" (ภาษาไทย)
    - Is it necessary to eat? Please reply "I recommend eating because there are only snacks and drinks available at the event." (English)
"""

contents = [
    Content(
        role="user",
        parts=[
            Part.from_uri(
                file_uri="gs://knowledge_chatbot/recepi_update_format.txt",
                mime_type="text/plain",
            ),
            Part.from_uri(
                file_uri="gs://knowledge_chatbot/Agenda_Speaker(Profile Summary).csv",
                mime_type="text/csv",
            ),
        ],
    )
]

content_cache = client.caches.create(
    model="gemini-2.5-flash",
    config=CreateCachedContentConfig(
        contents=contents,
        system_instruction=system_instruction,
        display_name="example-cache",
        ttl="86400s",
    ),
)
print(content_cache.name)
print(content_cache.usage_metadata)

## Remove cache

In [ ]:
for cache in client.caches.list():
    print("Name:", cache.name)
    print("Display name:", cache.display_name)
    print("Created:", cache.create_time)
    print("Expire time:", cache.expire_time)
    print("Usage:", cache.usage_metadata)
    print("------")

In [ ]:
for cache in client.caches.list():
    print("Deleting:", cache.name)
    client.caches.delete(name=cache.name)